In [4]:
# load attention map
import json
import os
import numpy as np
from copy import deepcopy
from PIL import Image
import matplotlib.pyplot as plt

from skimage.draw import polygon

original_data_path = './data/annotated_importance_heatmap'

ratio_th = 0.45

model_metrics_dict = {}


for model in ['phi3', 'internvl']:

    if model == 'phi3':
        region_overlap_result = './output/phi3_prob_results_dict.json'

        evaluated_results_path = "{TODO}" # TODO: add the path to the evaluated results of chart reasoning.

    elif model == 'internvl':
        region_overlap_result = './output/internvl_prob_results_dict.json'

        evaluated_results_path = "{TODO}" # TODO: add the path to the evaluated results of chart reasoning.

    region_overlap_dict = json.load(open(region_overlap_result, 'r'))

    sampled_ids = os.listdir(original_data_path)
    sampled_ids = [id for id in sampled_ids if id != '.DS_Store']

    
    matrix_dict = {
        'cover,correct': [],
        'cover,incorrect': [],
        'not_cover,correct': [],
        'not_cover,incorrect': [],
    }
    chart_type_matrix_dict = {
        'bar': deepcopy(matrix_dict),
        'bar_anno': deepcopy(matrix_dict)
    }

    

    for id in sampled_ids:

        for chart_type in ['bar', 'bar_anno']:        
            # print("Running on id:", id, "chart_type:", chart_type)
            response = region_overlap_dict[id][chart_type]['response']
            # print("Response:", response)
            # get the original evaluated results.
            json_path = os.path.join(evaluated_results_path, f'{id}.{chart_type}.grade.json')
            if not os.path.exists(json_path):
                print("Error: json_path not found for id:", id, "chart_type:", chart_type)
                continue
            with open(json_path, 'r') as f:
                original_evaluated_results = json.load(f)
            # print("Original Evaluated Results:", original_evaluated_results)
            # locate the retrieve value task.
            for task in original_evaluated_results:
                if task['task_type'] == 'Retrieve Value':
                    results = task['student_answer_correctness']
                    break
            if results.lower() != 'correct':
                results = 'incorrect'
            # print("Results:", results)
            if region_overlap_dict[id][chart_type]['ratio'] >= ratio_th:
                # print("Covered")
                chart_type_matrix_dict[chart_type]['cover,'+results].append((id, region_overlap_dict[id][chart_type]['ratio']))
            else:
                # print("Not Covered")
                chart_type_matrix_dict[chart_type]['not_cover,'+results].append((id, region_overlap_dict[id][chart_type]['ratio']))

        # print("="*100)
        # break
    
    model_metrics_dict[model] = deepcopy(chart_type_matrix_dict)    

    print("="*50, model, "="*50)
    print("Bar")
    print("Cover, Correct:", len(chart_type_matrix_dict['bar']['cover,correct']))
    print("Cover, Incorrect:", len(chart_type_matrix_dict['bar']['cover,incorrect']))
    print("Not Cover, Correct:", len(chart_type_matrix_dict['bar']['not_cover,correct']))
    print("Not Cover, Incorrect:", len(chart_type_matrix_dict['bar']['not_cover,incorrect']))
    print("Bar Anno")
    print("Cover, Correct:", len(chart_type_matrix_dict['bar_anno']['cover,correct']))
    print("Cover, Incorrect:", len(chart_type_matrix_dict['bar_anno']['cover,incorrect']))
    print("Not Cover, Correct:", len(chart_type_matrix_dict['bar_anno']['not_cover,correct']))
    print("Not Cover, Incorrect:", len(chart_type_matrix_dict['bar_anno']['not_cover,incorrect']))


================================================== phi3 ==================================================
Bar
Cover, Correct: 32
Cover, Incorrect: 59
Not Cover, Correct: 1
Not Cover, Incorrect: 8
Bar Anno
Cover, Correct: 63
Cover, Incorrect: 26
Not Cover, Correct: 2
Not Cover, Incorrect: 9
================================================== internvl ==================================================
Bar
Cover, Correct: 42
Cover, Incorrect: 52
Not Cover, Correct: 1
Not Cover, Incorrect: 5
Bar Anno
Cover, Correct: 68
Cover, Incorrect: 20
Not Cover, Correct: 3
Not Cover, Incorrect: 9
